In [15]:
import pandas as pd
import numpy as np 
import math 
from sklearn.ensemble import RandomForestClassifier


In [16]:
dp = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# age.describe() #decribe column for statiscial value
Y = dp['Survived'].copy()
X = dp[dp.columns.difference(['Survived'])].copy()

In [17]:
X.describe()
# Age has missing data. We fix it by filling missing data with mean

,Age,Fare,Parch,PassengerId,Pclass,SibSp
count,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,29.699118,32.204208,0.381594,446.000000,2.308642,0.523008
std,14.526497,49.693429,0.806057,257.353842,0.836071,1.102743
min,0.420000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,20.125000,7.910400,0.000000,223.500000,2.000000,0.000000
50%,28.000000,14.454200,0.000000,446.000000,3.000000,0.000000
75%,38.000000,31.000000,0.000000,668.500000,3.000000,1.000000
max,80.000000,512.329200,6.000000,891.000000,3.000000,8.000000


In [18]:
combined_data_set = [X,test]

In [19]:
# female is 0; male is 1
for data_set in combined_data_set:
    data_set['Sex'] = data_set['Sex'].replace({'male':1,'female':0})

In [20]:
# Embarked: S is 1; C is 2; Q is 3
for data_set in combined_data_set:
    data_set['Embarked'] = data_set['Embarked'].replace({'S':1,'C':2,'Q':3})
    data_set['Embarked'] = data_set['Embarked'].fillna(1)

In [21]:
# using feature extraction to convert Name into a new feature called 'Titles'
for data_set in combined_data_set:
    data_set['Titles'] = data_set['Name'].str.extract(' ([A-Za-z]+)\.', expand= False)

In [22]:
# we created a new feature called Titles: Mr: 0; Miss: 1; Mrs: 2; Others: 3
titles_mapping = {'Mr': 0, 'Miss': 1, 'Mrs':2}
for data_set in combined_data_set:
    data_set['Titles'] = data_set['Titles'].map(titles_mapping)
    data_set['Titles'] = data_set['Titles'].fillna(3)

In [23]:
# fill the missing ages with median based on their title
for data_set in combined_data_set:
    data_set['Age'].fillna(data_set.groupby('Titles')['Age'].transform('median'), inplace = True)

In [24]:
# fill the missing Fare based on the Passanger Class
for data_set in combined_data_set:
    data_set['Fare'].fillna(data_set.groupby('Pclass')['Fare'].transform('median'), inplace = True)
for data_set in combined_data_set:
    data_set.loc[data_set['Fare'] <= 17, 'Fare'] = 0
    data_set.loc[data_set['Fare'] > 17 & (data_set['Fare'] <= 30),'Fare'] = 1 
    data_set.loc[data_set['Fare'] > 30 & (data_set['Fare'] <= 100),'Fare'] = 2
    data_set.loc[data_set['Fare'] > 100 ,'Fare'] = 3

In [25]:
for data_set in combined_data_set:
    data_set.drop(['Cabin','Ticket','Name'], axis = 1, inplace = True)

In [26]:
model = RandomForestClassifier(n_estimators = 100)
model.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(Y, model.predict(X))

1.0

In [28]:
prediction = model.predict(test)
prediction = {'PassengerId': test['PassengerId'], 'Survived': prediction}
pd.DataFrame(prediction).to_csv('prediction.csv', index = False)